## Import the required libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [ ]:
# Create directory named "data" and unzip the downloaded movies data
!mkdir '/home/user/Desktop/data_engineering_task/data'
!unzip 'archive.zip' -d '/home/user/Desktop/data_engineering_task/data'

In [ ]:
# Check the unzipped data
%ls -l data/

## 2. Data exploration and  processing

The number of movies can be calculated from the movies_metadata.csv dataset

In [ ]:
movies_df = pd.read_csv('data/movies_metadata.csv', header=0)
movies_df.head(5)

In [ ]:
# Exploration of the dataset in search of the column for the counting of the films
movies_df.columns

In [ ]:
# Total rows in the movies metadata
len(movies_df)

There are 45466 rows in the movies_metadata.csv. `movies_df.columns` reveals that there are 2 possible id columns: 'id' and 'imdb_id'. Further inspection reveals that 'id' has more values and no NA values, while the index numbers in 'imdb_id' have 17 movies less. So, 'id' column might be appropriate for movie count and as an id column of the dataset.

In [ ]:
# There are 0 rows in 'id' column with NA values
movies_df['id'].isna().sum()

In [ ]:
# Here we have 17 movies not index by IMDB but existing in the movies database in Kaggle
movies_df['imdb_id'].isna().sum()

In [ ]:
# Here are these 17 movies
movies_df[movies_df['imdb_id'].isna()]

Also, `pd.read_csv` returns **DtypeWarning: Columns (10) have mixed types**. Columns (10) is the 'id' column. We will try to filter these non-alphanumeric symbols to see what's in there.

In [ ]:
# Filtering the non-alphanumeric symbols in the 'id' column
movies_df[~movies_df['id'].str.isalnum()]

Obviously, these are 3 rows (records) containinig movies with release dates, instead of numbers, in the 'id' column. These are 3 movies with neither much information about them, nor 'id' numbers, so it would be hard to use them in the further analysis. The easiest thing to do is to just drop them.

In [ ]:
# Dropping the non-alphanumeric rows
movies_df = movies_df[movies_df['id'].str.isalnum()]

Also, it's worthwile checking for duplicate rows. Here are they:

In [ ]:
# Duplicated rows in the 'id' column
movies_df[movies_df['id'].duplicated(keep=False)].sort_values('id')

In [ ]:
# Keeping only the rows with unique 'id' numbers
movies_df = movies_df.drop_duplicates(subset='id')

## 2. Putting it all together

Write all data processing operations as a single function

In [3]:
def data_processing(ratings_filepath, movies_filepath):
    """Read and clean up the data"""
    
    dtypes = {
        'genres': object,
        'release_date': str,
        'title': str,
    }
    
    # Read in the movies_metadata.csv
    movie_columns = ['genres', 'id', 'release_date', 'title']
    movies_df = pd.read_csv(movies_filepath, header=0 , sep=',', \
                            parse_dates=True, \
                            usecols=movie_columns, \
                            dtype=dtypes)
    
    # Dropping the non-alphanumeric rows
    movies_df = movies_df[movies_df['id'].str.isalnum()]
    
    # Keep only the rows with unique 'id' numbers
    movies_df = movies_df.drop_duplicates(subset='id')
    
    # Convert 'id' column to int data type
    movies_df['id'] = movies_df['id'].astype(int)
    
    # Set the 'id' column as movies_df index
    movies_df.set_index('id', inplace=True)  
    
    # Read in the ratings.csv
    columns = ['movieId', 'rating']
    ratings_df = pd.read_csv(ratings_filepath, \
                        usecols=columns, \
                        index_col='movieId', \
                         dtype={'movieId':int, \
                        'rating':float})
    
    return ratings_df, movies_df

ratings_filepath = 'data/ratings.csv'
movies_filepath = 'data/movies_metadata.csv'

ratings_df, movies_df = data_processing(ratings_filepath, movies_filepath)

## 3. Print the number of unique movies

In [ ]:
# Print the number of unique movies
len(movies_df)

## 4. Print the average rating of all the movies.

In [ ]:
def average_rating_loops(file_path):
    """Calculate the average rating of all movies"""
        
    # Define the variables
    chunk_size= 5000
    
    # Initialize variables for average rating calculation
    total_sum = 0
    total_count = 0
    
    # Read the file in chunks to prevent memory overloading
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        chunk_sum = chunk['rating'].sum()
        
        # Update the average values
        total_sum += chunk_sum
        total_count += len(chunk)
    
    # Calculate the average rating for all movies
    average_rating = total_sum/total_count
    
    # Print the average rating of all the movies
    print('Average movie rating: ' + str(np.round(average_rating, 2)))

file_path='data/ratings.csv'
average_rating_loops(file_path)

In [ ]:
# Another way is to load only the columns necessary for calculating the average rating of all movies

def average_rating(ratings):
    """Calculate the average rating of all movies"""
    
    average_rating = ratings_df['rating'].mean()
    print('Average movie rating: ' + str(np.round(average_rating, 2)))

average_rating(ratings_df)

## 5. Print the top 5 highest rated movies.

In [4]:
def top_5_movies(ratings, movies):
    """Print the top 5 highest rated movies"""
    
    # Group by 'movieId' and calculate the mean
    avg_ratings_df = ratings.groupby('movieId').mean('rating')
    
    # Merge the dataframes based on the index
    merged_df = pd.merge(avg_ratings_df, movies, left_index=True, right_index=True)
    
    # Get the top 5 movies
    top_5_movies = merged_df[['rating', 'title']].sort_values(by=['rating'], ascending=False).head(5)
    
    return top_5_movies

top5 = top_5_movies(ratings=ratings_df, movies=movies_df['title'])
top5

,rating,title
95977,5.0,The Man Behind The Gun
167666,5.0,Monster High: Escape from Skull Shores
130544,5.0,Palermo or Wolfsburg
129530,5.0,Brutal
164278,5.0,Harvey


## 6. Print the number of movies released each year.

In [ ]:
def movies_per_year(movies):
    """Print the number of movies per year"""
    
    # Extract the year component from 'release_date' and store into a new column 'release_year'
    movies['release_year'] = pd.to_datetime(movies['release_date'], errors='coerce').dt.year
    
    # Dropping the na in 'release_year'
    movies['release_year'] = movies['release_year'].dropna()
    
    # Count the number of movies released each year
    movies_per_year = movies['release_year'].value_counts().sort_index()
    
    # Create a movies_per_year_df DataFrame from the Pandas Series
    movies_per_year_df = pd.DataFrame({'Year': movies_per_year.index, 'Count': movies_per_year.values})
    
    # Cast 'Year' column to integer data type
    movies_per_year_df['Year'] = movies_per_year_df['Year'].astype(int)
    
    return movies_per_year_df
    
n_movies = movies_per_year(movies_df)

n_movies

## 7. Print the number of movies in each genre.

In [ ]:
import ast

def movies_per_genre(movies):
    """Print the number of movies in each genre"""
    
    # Convert the data in the 'genres' object to string
    # movies_df['genres'] = movies_df['genres'].apply(ast.literal_eval)
    if isinstance(movies_df['genres'].iloc[0], str):
        movies_df['genres'] = movies_df['genres'].apply(ast.literal_eval)

    # Extract the relevant information from the nested dictionaries
    movies_df['genre_names'] = movies_df['genres'].apply(lambda x: [genre['name'] for genre in x])
    
    # Count the number of movies per genre and convert to pandas.DataFrame
    movies_per_genre = movies_df['genre_names'].explode().value_counts()
    movies_per_genre.columns = ['Genre', 'Count']
    
    return movies_per_genre.to_frame().reset_index()

movies_per_genre(movies=movies_df)

### Save the dataset to a JSON file.

In [ ]:
top_5_movies.to_json()
movies_per_year.to_json()
movies_per_genre.to_json()

## 8. Movies analysis program using Pandas and OOP

In [1]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime, date


class MoviesAnalisys():
    """Data analysis of the movies dataset"""


    def __init__(self, movies_filepath, ratings_filepath):
        """Initialize attributes"""
        
        # Define the data types for each column
        dtypes_movies = {
            'genres': object,
            'release_date': str,
            'title': str,
        }
        
        # Read the movies_metadata.csv file
        movies_columns = ['genres', 'id', 'release_date', 'title']
        self.movies = pd.read_csv(movies_filepath, header=0, sep=',', \
                        parse_dates=True, \
                        usecols=movies_columns, \
                        dtype=dtypes_movies)
        
        # Drop the non-alphanumeric rows
        self.movies = self.movies[self.movies['id'].str.isalnum()]
        
        # Keep only the rows with unique 'id' numbers
        self.movies = self.movies.drop_duplicates(subset='id')
        
        # Convert 'id' column to int data type
        self.movies['id'] = self.movies['id'].astype(int)
        
        # Set the 'id' column as self.movies index
        self.movies.set_index('id', inplace=True)  
        
        ratings_columns = ['movieId', 'rating']
        self.ratings = pd.read_csv(ratings_filepath, \
                                usecols=ratings_columns, \
                                index_col='movieId', \
                                dtype={'movieId': int, 'rating':float})


    def count_movies(self):
        """Print the number of movies in the dataset"""

        print('Number of movies in the dataset: ' + str(len(self.movies)))


    def avg_rating(self):
        """Calculate the average rating of all movies"""
        
        average_rating = self.ratings['rating'].mean()
        print('Average movie rating: ' + str(np.round(average_rating, 2)))


    def top_5_movies(self):
        """Print the top 5 movies"""
        
        # Calculate the average ratign by movieId
        avg_ratings_df = self.ratings.groupby('movieId').apply('mean', 'rating')
        
        # Convert 'id' column to int data type
        # self.movies['id'] = self.movies['id'].astype(int)

        # Merge the dataframes based on ratingsmovieId and id
        merged_df = pd.merge(avg_ratings_df, self.movies, left_index=True, right_index=True)
        
        # Calculate the top 5 movies
        top_5_movies = merged_df[['rating', 'title']].sort_values('rating', ascending=False).head(5)
        
        # print('Top 5 movies:' + str(top_5_movies))
        
        return top_5_movies    

    
    def movies_per_genre(self):
        """Print the number of movies in each genre"""
        
        # Convert the data in the 'genres' object to string
        if isinstance(self.movies['genres'].iloc[0], str):
            self.movies['genres'] = self.movies['genres'].apply(ast.literal_eval)
    
        # Extract the relevant information from the nested dictionaries
        self.movies['genre_names'] = self.movies['genres'].apply(lambda x: [genre['name'] for genre in x])
        
        # Count the number of movies per genre and convert to pandas.DataFrame
        movies_per_genre = self.movies['genre_names'].explode().value_counts()
        movies_per_genre.columns = ['Genre', 'Count']
    
        return movies_per_genre.to_frame().reset_index()    

    
    def movies_per_year(self):
        """Print the number of movies per year"""
        
        # Extract the year component from 'release_date' and store into a new column 'release_year'
        self.movies['release_year'] = pd.to_datetime(self.movies['release_date'], errors='coerce').dt.year
        
        # Drop the `na` in 'release_year'
        self.movies['release_year'] = self.movies['release_year'].dropna()
        
        # Count the number of movies released each year
        movies_per_year = self.movies['release_year'].value_counts().sort_index()
        
        # Create a movies_per_year_df pandas.DataFrame from the pandas.Series
        movies_per_year_df = pd.DataFrame({'Year': movies_per_year.index, 'Count': movies_per_year.values})
        
        # Cast 'Year' column to integer data type
        movies_per_year_df['Year'] = movies_per_year_df['Year'].astype(int)
    
        return movies_per_year_df
    
    
    def save_to_json(self, result, filepath):
        """Save data to JSON"""
        
        result.to_json(filepath)

In [2]:
movies_filepath = 'data/movies_metadata.csv'
ratings_filepath = 'data/ratings.csv'

# Create an instance of the MoviesAnalysis class
movies_analysis = MoviesAnalisys(movies_filepath, ratings_filepath)

In [3]:
# Print the number of movies in the dataset
movies_count = movies_analysis.count_movies()

Number of movies in the dataset: 45433


In [4]:
# Calculate the average rating of all movies
average_rating = movies_analysis.avg_rating()

Average movie rating: 3.53


In [5]:
# Extract the top 5 movies
top_5_movies_result = movies_analysis.top_5_movies()
top_5_movies_result

,rating,title
95977,5.0,The Man Behind The Gun
167666,5.0,Monster High: Escape from Skull Shores
130544,5.0,Palermo or Wolfsburg
129530,5.0,Brutal
164278,5.0,Harvey


In [6]:
# Calculate the number of movies in each genre
movies_per_genre_result = movies_analysis.movies_per_genre()
movies_per_genre_result

,genre_names,count
0,Drama,20244
1,Comedy,13176
2,Thriller,7619
3,Romance,6730
4,Action,6592
5,Horror,4671
6,Crime,4304
7,Documentary,3930
8,Adventure,3490
9,Science Fiction,3044


In [8]:
# Calculate the number of movies released each year
movies_per_year_result = movies_analysis.movies_per_year()
movies_per_year_result

,Year,Count
0,1874,1
1,1878,1
2,1883,1
3,1887,1
4,1888,2
...,...,...
130,2015,1904
131,2016,1604
132,2017,532
133,2018,5


In [9]:
# Store some of the reulting dataframes as JSON files
top_5_movies_result_filepath = 'results_json/top_5_movies.json'
movies_per_genre_result_filepath = 'results_json/movies_per_genre.json'
movies_per_year_result_filepath = 'results_json/movies_per_year.json'

movies_analysis.save_to_json(top_5_movies_result, top_5_movies_result_filepath)
movies_analysis.save_to_json(movies_per_genre_result, movies_per_genre_result_filepath)
movies_analysis.save_to_json(movies_per_year_result, movies_per_year_result_filepath)